# 서울 열린데이터 아파트 가격 공공데이터 포털

In [15]:
import pandas as pd
import requests
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 설정
SERVICE_KEY = '4e506f62416a686a3131355164764c6f'
BASE_URL = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV'
NUM_PAGES = 11  # 1000건 * 11 = 최대 11,000건

def fetch_page(j):
    start_idx = 1 + (j - 1) * 1000
    end_idx = j * 1000
    url = f'{BASE_URL}/{start_idx}/{end_idx}'
    try:
        response = requests.get(url)
        content = response.json()
        rows = content['tbLnOpendataRtmsV'].get('row', [])
        return pd.DataFrame(rows)
    except Exception as e:
        print(f"[에러] 페이지 {j} 요청 실패: {e}")
        return pd.DataFrame()

results = []
with ThreadPoolExecutor(max_workers=6) as executor:
    for df in tqdm(executor.map(fetch_page, range(1, NUM_PAGES + 1)), total=NUM_PAGES):
        results.append(df)

data = pd.concat(results, ignore_index=True)

print(data.head())

100%|███████████████████████████████████████████████████████████████████| 11/11 [00:33<00:00,  3.01s/it]

  RCPT_YR CGG_CD CGG_NM STDG_CD STDG_NM LOTNO_SE LOTNO_SE_NM   MNO   SNO  \
0    2025  11530    구로구   10200     구로동        1          대지  1130  0023   
1    2025  11290    성북구   13800     장위동        1          대지  0219  0314   
2    2025  11620    관악구   10100     봉천동        1          대지  1717  0000   
3    2025  11290    성북구   13800     장위동        1          대지  0230  0105   
4    2025  11305    강북구   10100     미아동        1          대지  0129  0008   

       BLDG_NM  ... THING_AMT ARCH_AREA  LAND_AREA   FLR  RGHT_SE RTRCN_DAY  \
0  골드마인(Ⅲ)오피스텔  ...      6000     14.40      16.96   3.0                      
1       더커먼즈37  ...     28500     26.85      20.00   4.0                      
2       관악푸르지오  ...     76000     84.75       0.00  14.0                      
3        동방빌리지  ...     23400     40.98      18.00  -1.0                      
4       MK프라디움  ...     51500     55.00      38.00   4.0                      

  ARCH_YR BLDG_USG DCLR_SE OPBIZ_RESTAGNT_SGG_NM  
0    2002     오피스

# 공공데이터포털

In [29]:
import requests
import xmltodict
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# 
SERVICE_KEY = 'SDEjyCFJI1ki5hZaMAvBGUsre6U38rZKkVug+DGju9aAUp9G7+1tQagaeWcG2trJmx1+Xe0JANCSgpGnLFuOPA=='
BASE_URL = "http://apis.data.go.kr/1613000/RTMSDataSvcAptTradeDev/getRTMSDataSvcAptTradeDev"

# 지역 코드 (서울: 11110 ~ 전국 주요 코드, 일부만 예시. 전체 원할 시 파일 제공 가능)
REGION_CODES = [
    "11110", "11200", "11305",  # 서울 일부
    "26110", "26260",           # 부산 일부
    "41110", "41170",           # 경기 일부
    "42110",                   # 강원
    "43110",                   # 충북
    "44130",                   # 충남
    "45110",                   # 전북
    "46110",                   # 전남
    "47111",                   # 대구
    "48121",                   # 경남
    "50110"                    # 제주
]

# 월 리스트
months = [f"202501", "202502", "202503", "202504", "202505", "202506"]

# 수집 함수
def fetch_data(lawd_cd, deal_ymd):
    params = {
        "serviceKey": SERVICE_KEY,
        "LAWD_CD": lawd_cd,
        "DEAL_YMD": deal_ymd,
        "numOfRows": "1000",
        "pageNo": "1"
    }
    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        data = xmltodict.parse(response.text)
        items = data['response']['body']['items']
        if not items:
            return pd.DataFrame()
        item = items['item']
        if isinstance(item, list):
            return pd.DataFrame(item)
        else:
            return pd.DataFrame([item])
    except Exception as e:
        print(f"[오류] {lawd_cd}-{deal_ymd}: {e}")
        return pd.DataFrame()

# 병렬 처리 수집
results = []
with ThreadPoolExecutor(max_workers=12) as executor:
    tasks = []
    for ym in months:
        for code in REGION_CODES:
            tasks.append((code, ym))
    for df in tqdm(executor.map(lambda x: fetch_data(*x), tasks), total=len(tasks)):
        if not df.empty:
            results.append(df)

# 데이터 병합 및 정제
final_df = pd.concat(results, ignore_index=True)
print(final_df.head())


100%|███████████████████████████████████████████████████████████████████| 90/90 [00:01<00:00, 66.93it/s]

  aptDong      aptNm      aptSeq bonbun bubun buildYear buyerGbn cdealDay  \
0     102    인왕산아이파크  11110-2212   0060  0000      2008       개인     None   
1     105         현대    11110-90   0082  0000      2000       개인     None   
2     102      창신쌍용1    11110-37   0702  0000      1992       개인     None   
3     202  인왕산2차아이파크  11110-2417   0088  0000      2015       개인     None   
4   이화에수풀      이화에수풀  11110-2359   0195  0010      2014       법인     None   

  cdealType dealAmount  ... roadNmBonbun roadNmBubun roadNmCd roadNmSeq  \
0      None    115,000  ...        00009       00000  4100482        01   
1      None    139,950  ...        00246       00020  3000008        06   
2      None     69,500  ...        00019       00000  4100065        01   
3      None    111,500  ...        00034       00000  4100482        01   
4      None     17,700  ...        00047       00000  3100002        05   

  roadNmSggCd roadNmbCd  sggCd slerGbn  umdCd umdNm  
0       11110         0  11110  